In [8]:
from nyx_space.cosmic import Cosm, Orbit, Spacecraft
from nyx_space.mission_design import (
    propagate, SpacecraftDynamics, StateParameter
)
from nyx_space.monte_carlo import generate_orbits
from nyx_space.time import Epoch
import pandas as pd
from plotly.offline import init_notebook_mode


%load_ext autoreload
%autoreload 2


init_notebook_mode(connected=True)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# create the generic orbit
cosm = Cosm.de438()
eme2k = cosm.frame("EME2000")
e0 = Epoch('2023-01-01 00:00:00.0000 UTC')
e1 = Epoch('2023-01-01 01:00:00.0000 UTC')
orbit_params = dict(
    sma_altitude_km=500,
    ecc=1e-4,
    inc_deg=90,
    raan_deg=35,
    aop_deg=65,
    ta_deg=590,
    epoch=e0,
    frame=eme2k,
)
template_orbit = Orbit.from_keplerian_altitude(**orbit_params)


In [10]:
# create 100 orbits with varied SMA, Ecc, and Inc
orbits = generate_orbits(
    template_orbit,
    [
        (StateParameter.SMA, 200),
        (StateParameter.Inclination, 30),
        (StateParameter.RAAN, 30),
    ],
    100,
    kind='abs',
)


In [11]:
# create trajectories from orbits so we can save them as parquet files.
# this is the only way I can figure out how to convert a trajectory to dataframe
traj_files = []
dynamics = SpacecraftDynamics.load('simpledynamics.yaml')
for index, orbit in enumerate(orbits):
    sc = Spacecraft(orbit, 100, 0)
    _, traj = propagate(sc, dynamics, duration=e0.timedelta(e1))
    fname = f'tmp/{index}.parquet'
    traj.to_parquet(fname)
    traj_files.append(fname)


Removed unavailable field `isp` from trajectory export
Removed unavailable field `thrust` from trajectory export
Removed unavailable field `isp` from trajectory export
Removed unavailable field `thrust` from trajectory export
Removed unavailable field `isp` from trajectory export
Removed unavailable field `thrust` from trajectory export
Removed unavailable field `isp` from trajectory export
Removed unavailable field `thrust` from trajectory export
Removed unavailable field `isp` from trajectory export
Removed unavailable field `thrust` from trajectory export
Removed unavailable field `isp` from trajectory export
Removed unavailable field `thrust` from trajectory export
Removed unavailable field `isp` from trajectory export
Removed unavailable field `thrust` from trajectory export
Removed unavailable field `isp` from trajectory export
Removed unavailable field `thrust` from trajectory export
Removed unavailable field `isp` from trajectory export
Removed unavailable field `thrust` from t

In [12]:
from nyx_space.plots.traj import plot_orbit_elements


plot_orbit_elements(pd.read_parquet(traj_files[0]), title='')


In [ ]:
from nyx_space.plots import plot_traj

traj_dfs = []
for index, fname in enumerate(traj_files):
    traj_dfs.append(pd.read_parquet(fname))

plot_traj(traj_dfs, title='monte carlo', html_out='../docs/example.md')
